In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras

# Transfer learning with keras

## Use pretrained fashion mnist model on binary classification

Steps:
1. Load pretrained model

In [2]:
modelA = tf.keras.models.load_model('my_model_A')

2. form a modelB with all layers from modelA except output<br>
add output layer

In [3]:
modelB_on_A = keras.models.Sequential(modelA.layers[:-1])
modelB_on_A.add(keras.layers.Dense(name='output',units=1,activation='sigmoid'))
modelB_on_A.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 265,701
Trainable params: 265,701
Non-trainable params: 0
_________________________________________________________________


3. Clone pretrained model to back up (not necessary)

In [4]:
modelA_clone = keras.models.clone_model(modelA)
modelA_clone.set_weights(modelA.get_weights())
modelA_clone.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


4. Freeze added layers of modelB for some epochs to avoid overriding of weights<br>
After few epochs when output layers' weights are headed in right direction we can unfreeze

In [5]:
for layer in modelB_on_A.layers[:-1]:
    layer.trainable = False

In [6]:
# create training data for new task - binary classification

In [7]:
(X_train,y_train), (X_test,y_test) = fashion_mnist = keras.datasets.fashion_mnist.load_data()
category1_indices = np.where(y_test==0)[0]
category2_indices = np.where(y_test==1)[0]
X_cat1 = X_test[category1_indices][:100]
X_cat2 = X_test[category2_indices][:100]
X__train = np.concatenate([X_cat1,X_cat2],axis=0)
y_cat1 = y_test[category1_indices][:100]
y_cat2 = y_test[category2_indices][:100]
y__train = np.concatenate([y_cat1,y_cat2],axis=0)
X__test = np.concatenate([X_test[category1_indices][100:120],
                         X_test[category2_indices][100:120]])
y__test = np.concatenate([y_test[category1_indices][100:120],
                         y_test[category2_indices][100:120]])

In [8]:
# compile the model

In [9]:
modelB_on_A.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

5. Fit the model with less epochs to let output layer adjust weights with freezing

In [10]:
history = modelB_on_A.fit(X__train,y__train,epochs=4)

Epoch 1/4
7/7 [==============================] - 0s 2ms/step - loss: 1481.4801 - accuracy: 0.8400
Epoch 2/4
7/7 [==============================] - 0s 2ms/step - loss: 1.4860 - accuracy: 0.9950
Epoch 3/4
7/7 [==============================] - 0s 2ms/step - loss: 3.1019 - accuracy: 0.9900
Epoch 4/4
7/7 [==============================] - 0s 3ms/step - loss: 0.0198 - accuracy: 0.9950


6. Now unfreeze the layers to train weights

In [11]:
for layer in modelB_on_A.layers[:-1]:
    layer.trainable = True

7. Retrain the model with lower learning rate and more epochs

In [12]:
optim = keras.optimizers.SGD(lr=1e-4)

In [13]:
modelB_on_A.compile(loss='binary_crossentropy',optimizer=optim,metrics=['accuracy'])

In [14]:
history = modelB_on_A.fit(X__train,y__train,epochs=16)

Epoch 1/16
7/7 [==============================] - 0s 2ms/step - loss: 3.4074 - accuracy: 0.9950
Epoch 2/16
7/7 [==============================] - 0s 2ms/step - loss: 1598.7640 - accuracy: 0.7400
Epoch 3/16
7/7 [==============================] - 0s 3ms/step - loss: 13.1804 - accuracy: 0.9650
Epoch 4/16
7/7 [==============================] - 0s 2ms/step - loss: 3.2912 - accuracy: 0.9900
Epoch 5/16
7/7 [==============================] - 0s 3ms/step - loss: 4.8868e-10 - accuracy: 1.0000
Epoch 6/16
7/7 [==============================] - 0s 2ms/step - loss: 4.8868e-10 - accuracy: 1.0000
Epoch 7/16
7/7 [==============================] - 0s 3ms/step - loss: 4.8871e-10 - accuracy: 1.0000
Epoch 8/16
7/7 [==============================] - 0s 2ms/step - loss: 4.8871e-10 - accuracy: 1.0000
Epoch 9/16
7/7 [==============================] - 0s 3ms/step - loss: 4.8871e-10 - accuracy: 1.0000
Epoch 10/16
7/7 [==============================] - 0s 3ms/step - loss: 4.8871e-10 - accuracy: 1.0000
Epoch 11/16

# Very Well Done, you performed transfer learning for a task !!! 

# :D

# I am Great HaHa :p